# Player Statistical Models
- The goal of this notebook is to determine the best way to create predictions of strokes gained metrics for the upcoming tournament based on past observances

In [1]:
import sqlite3
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import (mean_squared_error, mean_absolute_error, r2_score)
from xgboost import XGBRegressor
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import pandas as pd
import numpy as np

conn = sqlite3.connect('/Users/nickospelt/Documents/App_Projects/PGA_Score_Predictor/Data/PGA_SQL_DB/PGA.db')

## Import Tournament Data

In [12]:
model_data_df = pd.read_csv('/Users/nickospelt/Documents/App_Projects/PGA_Score_Predictor/Model Development/model_data.csv')
model_data_df.sort_values(by=['PLAYER_NAME', 'TOURNAMENT_DATE'], ascending=[True, True])

,TOURNAMENT_NAME,TOURNAMENT_DATE,PLAYER_NAME,PAR,LENGTH,ELEVATION,R1_TEMP,R1_PRECIP,R1_WIND_SPEED,R2_TEMP,...,PREV_SG_ATG,PREV_3_AVG_SG_ATG,HL_50_SG_ATG,HL_100_SG_ATG,HL_200_SG_ATG,PREV_TOT_SCORE_TO_PAR,PREV_3_AVG_TOT_SCORE_TO_PAR,HL_50_TOT_SCORE_TO_PAR,HL_100_TOT_SCORE_TO_PAR,HL_200_TOT_SCORE_TO_PAR
0,2017 THE PLAYERS Championship,2017-05-11,Aaron Baddeley,72,7189,-0.928824,1.477375,-0.338412,-0.605286,1.538345,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017 Sanderson Farms Championship,2017-10-26,Aaron Baddeley,72,7440,-0.342815,-1.168057,-0.338412,-0.335730,-0.685256,...,-0.210268,-0.210268,-0.210268,-0.210268,-0.210268,1.394366,1.394366,1.394366,1.394366,1.394366
2,2018 Valero Texas Open,2018-04-19,Aaron Baddeley,72,7435,0.369719,0.395153,-0.338412,1.461306,-0.278322,...,0.241306,0.015519,0.201228,0.133898,0.079454,-0.750000,0.322183,-0.559684,-0.239956,0.018576
3,2018 U.S. Open,2018-06-14,Aaron Baddeley,70,7440,-0.895528,-0.232804,-0.322663,2.157658,-0.932323,...,0.957887,0.329642,0.890983,0.726660,0.554320,-3.661972,-1.005869,-3.387666,-2.701688,-1.971064
4,2019 Genesis Open,2019-02-14,Aaron Baddeley,71,7322,-0.262905,-1.889539,5.969095,1.281603,-2.530990,...,-0.288758,0.303478,0.106957,0.204029,0.220233,-0.500000,-1.637324,-1.468596,-1.568490,-1.388124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5287,2020 Rocket Mortgage Classic,2020-07-02,Zack Sucher,72,7340,0.249853,1.370488,-0.338412,-0.380656,1.930746,...,0.071125,-0.193099,-0.095396,-0.142453,-0.167539,4.472222,2.785407,3.409149,3.108737,2.948585
5288,2018 Valero Texas Open,2018-04-19,Zecheng Dou,72,7435,0.369719,0.395153,-0.338412,1.461306,-0.278322,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5289,2022 Cadence Bank Houston Open,2022-11-10,Zecheng Dou,70,7412,-0.828936,0.582203,-0.338412,-0.245879,0.157678,...,-0.202113,-0.202113,-0.202113,-0.202113,-0.202113,3.338028,3.338028,3.338028,3.338028,3.338028
5290,2023 Valspar Championship,2023-03-16,Zecheng Dou,71,7340,-0.862232,-0.887481,-0.338412,0.203381,-0.103922,...,0.029896,-0.086109,0.029896,0.029893,0.029177,14.074627,8.706328,14.074627,14.074523,14.041366


## Strokes Gained Putting

In [17]:
sg_putt_query = """WITH 
TOURNAMENT_AVERAGES AS (
    SELECT TOURNAMENT_NAME,
        AVG(SG_PUTT) AS TOURN_AVG_SG_PUTT,
        AVG(SG_OFF_THE_TEE) AS TOURN_AVG_SG_OFF_THE_TEE,
        AVG(SG_APPROACH) AS TOURN_AVG_SG_APPROACH,
        AVG(SG_AROUND_THE_GREEN) AS TOURN_AVG_SG_AROUND_THE_GREEN,
        AVG(TOTAL_SCORE_TO_PAR) AS TOURN_AVG_TOTAL_SCORE_TO_PAR
    FROM PLAYER_TOURNAMENT_RESULTS_V3
    GROUP BY 1
),
ADJUSTED_METRICS AS (
    SELECT PLAYER_TOURNAMENT_RESULTS_V3.TOURNAMENT_NAME, TOURNAMENT_DATE, PLAYER_NAME,
        SG_PUTT, TOURN_AVG_SG_PUTT, SG_PUTT - TOURN_AVG_SG_PUTT AS ADJ_SG_PUTT
    FROM PLAYER_TOURNAMENT_RESULTS_V3
    INNER JOIN TOURNAMENT_AVERAGES ON PLAYER_TOURNAMENT_RESULTS_V3.TOURNAMENT_NAME = TOURNAMENT_AVERAGES.TOURNAMENT_NAME
    ORDER BY PLAYER_NAME, PLAYER_TOURNAMENT_RESULTS_V3.TOURNAMENT_DATE
),
LAGGED_ADJUSTED_METRICS AS (
    SELECT PLAYER_NAME, TOURNAMENT_DATE, ADJ_SG_PUTT,
        LAG(TOURNAMENT_DATE, 1) OVER (PARTITION BY PLAYER_NAME ORDER BY TOURNAMENT_DATE) AS P1_DATE,
        LAG(ADJ_SG_PUTT, 1) OVER (PARTITION BY PLAYER_NAME ORDER BY TOURNAMENT_DATE) AS P1_ADJ_SG,
        LAG(TOURNAMENT_DATE, 2) OVER (PARTITION BY PLAYER_NAME ORDER BY TOURNAMENT_DATE) AS P2_DATE,
        LAG(ADJ_SG_PUTT, 2) OVER (PARTITION BY PLAYER_NAME ORDER BY TOURNAMENT_DATE) AS P2_ADJ_SG,
        LAG(TOURNAMENT_DATE, 3) OVER (PARTITION BY PLAYER_NAME ORDER BY TOURNAMENT_DATE) AS P3_DATE,
        LAG(ADJ_SG_PUTT, 3) OVER (PARTITION BY PLAYER_NAME ORDER BY TOURNAMENT_DATE) AS P3_ADJ_SG,
        LAG(TOURNAMENT_DATE, 4) OVER (PARTITION BY PLAYER_NAME ORDER BY TOURNAMENT_DATE) AS P4_DATE,
        LAG(ADJ_SG_PUTT, 4) OVER (PARTITION BY PLAYER_NAME ORDER BY TOURNAMENT_DATE) AS P4_ADJ_SG,
        LAG(TOURNAMENT_DATE, 5) OVER (PARTITION BY PLAYER_NAME ORDER BY TOURNAMENT_DATE) AS P5_DATE,
        LAG(ADJ_SG_PUTT, 5) OVER (PARTITION BY PLAYER_NAME ORDER BY TOURNAMENT_DATE) AS P5_ADJ_SG,
        LAG(TOURNAMENT_DATE, 6) OVER (PARTITION BY PLAYER_NAME ORDER BY TOURNAMENT_DATE) AS P6_DATE,
        LAG(ADJ_SG_PUTT, 6) OVER (PARTITION BY PLAYER_NAME ORDER BY TOURNAMENT_DATE) AS P6_ADJ_SG,
        LAG(TOURNAMENT_DATE, 7) OVER (PARTITION BY PLAYER_NAME ORDER BY TOURNAMENT_DATE) AS P7_DATE,
        LAG(ADJ_SG_PUTT, 7) OVER (PARTITION BY PLAYER_NAME ORDER BY TOURNAMENT_DATE) AS P7_ADJ_SG,
        LAG(TOURNAMENT_DATE, 8) OVER (PARTITION BY PLAYER_NAME ORDER BY TOURNAMENT_DATE) AS P8_DATE,
        LAG(ADJ_SG_PUTT, 8) OVER (PARTITION BY PLAYER_NAME ORDER BY TOURNAMENT_DATE) AS P8_ADJ_SG,
        LAG(TOURNAMENT_DATE, 9) OVER (PARTITION BY PLAYER_NAME ORDER BY TOURNAMENT_DATE) AS P9_DATE,
        LAG(ADJ_SG_PUTT, 9) OVER (PARTITION BY PLAYER_NAME ORDER BY TOURNAMENT_DATE) AS P9_ADJ_SG,
        LAG(TOURNAMENT_DATE, 10) OVER (PARTITION BY PLAYER_NAME ORDER BY TOURNAMENT_DATE) AS P10_DATE,
        LAG(ADJ_SG_PUTT, 10) OVER (PARTITION BY PLAYER_NAME ORDER BY TOURNAMENT_DATE) AS P10_ADJ_SG
    FROM ADJUSTED_METRICS
)

SELECT *
FROM LAGGED_ADJUSTED_METRICS
WHERE P10_ADJ_SG IS NOT NULL
ORDER BY PLAYER_NAME, TOURNAMENT_DATE"""

sg_putt_df = pd.read_sql_query(sg_putt_query, conn)
sg_putt_df

,PLAYER_NAME,TOURNAMENT_DATE,ADJ_SG_PUTT,P1_DATE,P1_ADJ_SG,P2_DATE,P2_ADJ_SG,P3_DATE,P3_ADJ_SG,P4_DATE,...,P6_DATE,P6_ADJ_SG,P7_DATE,P7_ADJ_SG,P8_DATE,P8_ADJ_SG,P9_DATE,P9_ADJ_SG,P10_DATE,P10_ADJ_SG
0,Aaron Rai,2023-10-12,-0.825993,2023-06-29,0.289042,2023-04-13,-0.778932,2023-03-09,-0.253573,2023-02-09,...,2022-06-23,-0.459246,2022-06-09,-0.769111,2022-05-12,0.244485,2022-03-31,0.464116,2021-02-25,0.657441
1,Aaron Rai,2024-03-14,-0.436827,2023-10-12,-0.825993,2023-06-29,0.289042,2023-04-13,-0.778932,2023-03-09,...,2022-11-10,0.853007,2022-06-23,-0.459246,2022-06-09,-0.769111,2022-05-12,0.244485,2022-03-31,0.464116
2,Aaron Rai,2024-04-04,0.257973,2024-03-14,-0.436827,2023-10-12,-0.825993,2023-06-29,0.289042,2023-04-13,...,2023-02-09,-0.391812,2022-11-10,0.853007,2022-06-23,-0.459246,2022-06-09,-0.769111,2022-05-12,0.244485
3,Aaron Rai,2024-05-16,-0.787813,2024-04-04,0.257973,2024-03-14,-0.436827,2023-10-12,-0.825993,2023-06-29,...,2023-03-09,-0.253573,2023-02-09,-0.391812,2022-11-10,0.853007,2022-06-23,-0.459246,2022-06-09,-0.769111
4,Aaron Rai,2024-05-30,-1.233566,2024-05-16,-0.787813,2024-04-04,0.257973,2024-03-14,-0.436827,2023-10-12,...,2023-04-13,-0.778932,2023-03-09,-0.253573,2023-02-09,-0.391812,2022-11-10,0.853007,2022-06-23,-0.459246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2199,Zach Johnson,2023-02-09,0.379188,2022-11-10,0.041507,2022-03-31,0.678116,2022-03-03,0.916964,2021-05-06,...,2020-09-17,2.010075,2020-08-20,-0.444843,2020-07-16,-0.749382,2020-06-11,0.118646,2020-03-05,0.831337
2200,Zach Johnson,2023-04-13,-0.747932,2023-02-09,0.379188,2022-11-10,0.041507,2022-03-31,0.678116,2022-03-03,...,2021-03-11,1.257371,2020-09-17,2.010075,2020-08-20,-0.444843,2020-07-16,-0.749382,2020-06-11,0.118646
2201,Zach Johnson,2023-05-18,-0.108652,2023-04-13,-0.747932,2023-02-09,0.379188,2022-11-10,0.041507,2022-03-31,...,2021-05-06,1.154993,2021-03-11,1.257371,2020-09-17,2.010075,2020-08-20,-0.444843,2020-07-16,-0.749382
2202,Zach Johnson,2023-06-29,0.968792,2023-05-18,-0.108652,2023-04-13,-0.747932,2023-02-09,0.379188,2022-11-10,...,2022-03-03,0.916964,2021-05-06,1.154993,2021-03-11,1.257371,2020-09-17,2.010075,2020-08-20,-0.444843


### Attempt to model these trends